# 机器学习工程师纳米学位
## 监督学习
## 项目 2: 搭建一个学生干预系统

欢迎来到机器学习工程师纳米学位的第二个项目！在此文件中，有些示例代码已经提供给你，但你还需要实现更多的功能让项目成功运行。除非有明确要求，你无须修改任何已给出的代码。以**'练习'**开始的标题表示接下来的代码部分中有你必须要实现的功能。每一部分都会有详细的指导，需要实现的部分也会在注释中以**'TODO'**标出。请仔细阅读所有的提示！

除了实现代码外，你还**必须**回答一些与项目和你的实现有关的问题。每一个需要你回答的问题都会以**'问题 X'**为标题。请仔细阅读每个问题，并且在问题后的**'回答'**文字框中写出完整的答案。我们将根据你对问题的回答和撰写代码所实现的功能来对你提交的项目进行评分。

>**提示：**Code 和 Markdown 区域可通过 **Shift + Enter** 快捷键运行。此外，Markdown可以通过双击进入编辑模式。

### 问题 1 - 分类 vs. 回归
*在这个项目中你的任务是找出那些如果不给予帮助，最终可能无法毕业的学生。你觉得这个问题是哪种类型的监督学习问题，是分类问题还是回归问题？为什么？*

**答案: ** 由于学生可能处于能够毕业和不能毕业两种状态中的一种，因此我认为这是一个分类问题。

## 分析数据
运行下面区域的代码以载入学生数据集，以及一些此项目所需的Python库。注意数据集的最后一列`'passed'`是我们的预测的目标（表示学生是毕业了还是没有毕业），其他的列是每个学生的属性。

In [1]:
# 载入所需要的库
import numpy as np
import pandas as pd
from time import time
from sklearn.metrics import f1_score

# 载入学生数据集
student_data = pd.read_csv("student-data.csv")
print "Student data read successfully!"

Student data read successfully!


In [2]:
student_data.head()

,school,sex,age,address,famsize,Pstatus,Medu,Fedu,Mjob,Fjob,...,internet,romantic,famrel,freetime,goout,Dalc,Walc,health,absences,passed
0,GP,F,18,U,GT3,A,4,4,at_home,teacher,...,no,no,4,3,4,1,1,3,6,no
1,GP,F,17,U,GT3,T,1,1,at_home,other,...,yes,no,5,3,3,1,1,3,4,no
2,GP,F,15,U,LE3,T,1,1,at_home,other,...,yes,no,4,3,2,2,3,3,10,yes
3,GP,F,15,U,GT3,T,4,2,health,services,...,yes,yes,3,2,2,1,1,5,2,yes
4,GP,F,16,U,GT3,T,3,3,other,other,...,no,no,4,3,2,1,2,5,4,yes


### 练习: 分析数据
我们首先通过调查数据，以确定有多少学生的信息，并了解这些学生的毕业率。在下面的代码单元中，你需要完成如下的运算：
- 学生的总数， `n_students`。
- 每个学生的特征总数， `n_features`。
- 毕业的学生的数量， `n_passed`。
- 未毕业的学生的数量， `n_failed`。
- 班级的毕业率， `grad_rate`， 用百分数表示(%)。


In [3]:
# TODO： 计算学生的数量
n_students = len(student_data.count(axis=1))

# TODO： 计算特征数量
n_features = len(student_data.count()) - 1

# TODO： 计算通过的学生数
n_passed = len(student_data['passed'][student_data['passed'] == 'yes'])

# TODO： 计算未通过的学生数
n_failed = len(student_data['passed'][student_data['passed'] == 'no'])

# TODO： 计算通过率
grad_rate = float(n_passed)/(float(n_passed) + float(n_failed)) * 100

# 输出结果
print "Total number of students: {}".format(n_students)
print "Number of features: {}".format(n_features)
print "Number of students who passed: {}".format(n_passed)
print "Number of students who failed: {}".format(n_failed)
print "Graduation rate of the class: {:.2f}%".format(grad_rate)

Total number of students: 395
Number of features: 30
Number of students who passed: 265
Number of students who failed: 130
Graduation rate of the class: 67.09%


## 数据准备
在这个部分中，我们将要为建模、训练和测试准备数据
### 识别特征和目标列
你获取的数据中通常都会包含一些非数字的特征，这会导致一些问题，因为大多数的机器学习算法都会期望输入数字特征进行计算。

运行下面的代码单元将学生数据分成特征和目标列看一看他们中是否有非数字特征。

In [4]:
# 提取特征列
feature_cols = list(student_data.columns[:-1])

# 提取目标列 ‘passed’
target_col = student_data.columns[-1] 

# 显示列的列表
print "Feature columns:\n{}".format(feature_cols)
print "\nTarget column: {}".format(target_col)

# 将数据分割成特征数据和目标数据（即X_all 和 y_all）
X_all = student_data[feature_cols]
y_all = student_data[target_col]

# 通过打印前5行显示特征信息
print "\nFeature values:"
print X_all.head()

Feature columns:
['school', 'sex', 'age', 'address', 'famsize', 'Pstatus', 'Medu', 'Fedu', 'Mjob', 'Fjob', 'reason', 'guardian', 'traveltime', 'studytime', 'failures', 'schoolsup', 'famsup', 'paid', 'activities', 'nursery', 'higher', 'internet', 'romantic', 'famrel', 'freetime', 'goout', 'Dalc', 'Walc', 'health', 'absences']

Target column: passed

Feature values:
  school sex  age address famsize Pstatus  Medu  Fedu     Mjob      Fjob  \
0     GP   F   18       U     GT3       A     4     4  at_home   teacher   
1     GP   F   17       U     GT3       T     1     1  at_home     other   
2     GP   F   15       U     LE3       T     1     1  at_home     other   
3     GP   F   15       U     GT3       T     4     2   health  services   
4     GP   F   16       U     GT3       T     3     3    other     other   

    ...    higher internet  romantic  famrel  freetime goout Dalc Walc health  \
0   ...       yes       no        no       4         3     4    1    1      3   
1   ...       

### 预处理特征列

正如你所见，我们这里有几个非数值的列需要做一定的转换！它们中很多是简单的`yes`/`no`，比如`internet`。这些可以合理地转化为`1`/`0`（二元值，binary）值。

其他的列，如`Mjob`和`Fjob`，有两个以上的值，被称为_分类变量（categorical variables）_。处理这样的列的推荐方法是创建和可能值一样多的列（如：`Fjob_teacher`，`Fjob_other`，`Fjob_services`等），然后将其中一个的值设为`1`另外的设为`0`。

这些创建的列有时候叫做 _虚拟变量（dummy variables）_，我们将用[`pandas.get_dummies()`](http://pandas.pydata.org/pandas-docs/stable/generated/pandas.get_dummies.html?highlight=get_dummies#pandas.get_dummies)函数来完成这个转换。运行下面代码单元的代码来完成这里讨论的预处理步骤。

In [5]:
def preprocess_features(X):
    ''' 预处理学生数据，将非数字的二元特征转化成二元值（0或1），将分类的变量转换成虚拟变量
    '''
    
    # 初始化一个用于输出的DataFrame
    output = pd.DataFrame(index = X.index)

    # 查看数据的每一个特征列
    for col, col_data in X.iteritems():
        
        # 如果数据是非数字类型，将所有的yes/no替换成1/0
        if col_data.dtype == object:
            col_data = col_data.replace(['yes', 'no'], [1, 0])

        # 如果数据类型是类别的（categorical），将它转换成虚拟变量
        if col_data.dtype == object:
            # 例子: 'school' => 'school_GP' and 'school_MS'
            col_data = pd.get_dummies(col_data, prefix=col)
        
        # 收集转换后的列
        output = output.join(col_data)
    
    return output

X_all = preprocess_features(X_all)
print "Processed feature columns ({} total features):\n{}".format(len(X_all.columns), list(X_all.columns))

Processed feature columns (48 total features):
['school_GP', 'school_MS', 'sex_F', 'sex_M', 'age', 'address_R', 'address_U', 'famsize_GT3', 'famsize_LE3', 'Pstatus_A', 'Pstatus_T', 'Medu', 'Fedu', 'Mjob_at_home', 'Mjob_health', 'Mjob_other', 'Mjob_services', 'Mjob_teacher', 'Fjob_at_home', 'Fjob_health', 'Fjob_other', 'Fjob_services', 'Fjob_teacher', 'reason_course', 'reason_home', 'reason_other', 'reason_reputation', 'guardian_father', 'guardian_mother', 'guardian_other', 'traveltime', 'studytime', 'failures', 'schoolsup', 'famsup', 'paid', 'activities', 'nursery', 'higher', 'internet', 'romantic', 'famrel', 'freetime', 'goout', 'Dalc', 'Walc', 'health', 'absences']


### 实现: 将数据分成训练集和测试集
现在我们已经将所有的 _分类的（categorical）_ 特征转换成数值了。下一步我们将把数据（包括特征和对应的标签数据）分割成训练集和测试集。在下面的代码单元中，你需要完成下列功能：
- 随机混洗切分数据(`X_all`, `y_all`) 为训练子集和测试子集。
  - 使用300个数据点作为训练集（约76%），使用95个数据点作为测试集（约24%）。
  - 如果可能的话，为你使用的函数设置一个`random_state`。
  - 将结果存储在`X_train`, `X_test`, `y_train`和 `y_test`中。

In [6]:
# TODO：在这里导入你可能需要使用的另外的功能
from sklearn.model_selection import train_test_split
# TODO：设置训练集的数量
num_train = 300

# TODO：设置测试集的数量
num_test = X_all.shape[0] - num_train

# TODO：把数据集混洗和分割成上面定义的训练集和测试集
X_train, X_test, y_train, y_test = train_test_split(X_all, y_all, test_size=0.24, random_state=233)
# X_train = None
# X_test = None
# y_train = None
# y_test = None

# 显示分割的结果
print "Training set has {} samples.".format(X_train.shape[0])
print "Testing set has {} samples.".format(X_test.shape[0])

Training set has 300 samples.
Testing set has 95 samples.


## 训练和评价模型
在这个部分，你将选择3个适合这个问题并且在`scikit-learn`中已有的监督学习的模型。首先你需要说明你选择这三个模型的原因，包括这些数据集有哪些特点，每个模型的优点和缺点各是什么。然后，你需要将这些模型用不同大小的训练集（100个数据点，200个数据点，300个数据点）进行训练，并用F<sub>1</sub>的值来衡量。你需要制作三个表，每个表要显示训练集大小，训练时间，预测时间，训练集上的F<sub>1</sub>值和测试集上的F<sub>1</sub>值（每个模型一个表）。

**这是目前** [`scikit-learn`](http://scikit-learn.org/stable/supervised_learning.html) **里有的监督学习模型，你可以从中选择:**
- Gaussian Naive Bayes (GaussianNB) 朴素贝叶斯
- Decision Trees 决策树
- Ensemble Methods (Bagging, AdaBoost, Random Forest, Gradient Boosting)
- K-Nearest Neighbors (KNeighbors)
- Stochastic Gradient Descent (SGDC)
- Support Vector Machines (SVM) 向量模型机
- Logistic Regression 逻辑回归

### 问题 2 - 应用模型
*列出三个适合这个问题的监督学习算法模型。每一个你选择的模型：*

- 描述一个该模型在真实世界的一个应用场景。（你需要为此做点研究，并给出你的引用出处）
- 这个模型的优势是什么？他什么情况下表现最好？
- 这个模型的缺点是什么？什么条件下它表现很差？
- 根据我们当前数据集的特点，为什么这个模型适合这个问题。

**回答: **
选用的三个模型为Support Vector Machines (SVM) 向量模型机模型，朴素贝叶斯模型，KNN模型。

在Wei Yu的论文Application of support vector machine modeling for prediction of common diseases: the case of diabetes and pre-diabetes（2010）中提到，Support Vector Machines (SVM) 向量模型机可以用于早期糖尿病的预测。论文作者采用1999-2004年美国National Health and Nutrition Examination Survey的数据作为原始数据，其中包括家族史，种族，年龄，身高，体重，腰围，高血压史等数据，采用SVM模型对早期糖尿病进行预测，结果体现出了83.5%的准确率。
SVM模型由于其最大化margin的特性，使得模型本身不易过拟合，且不易受到数据噪声的影响。通过改变SVM模型的核函数可以轻松解决线性和非线性的分类问题，也能轻松处理高维数据。SVM模型基于凸优化理论，可以解析出模型的全局最优解。在面对中等体量（50-100000）的数据时，通过调参和改变核函数的方法，SVM模型在绝大多数情况下表现良好。
SVM模型的缺点在于，模型在采用非线性核函数（如rbf）时其结果难以解释，且数据量太大（大于100000）时模型训练需要花费很长时间。另一方面，在分类数据极度不平衡时，例如一些子类样本量过小，也会使得SVM表现不佳。
当前数据集数据体量中等，各子类数据量能够满足模型训练需求，且数据集线性可分，因此可以使用Support Vector Machines (SVM) 向量模型机模型。

在Radha Vedala的论文An application of Naive Bayes classification for credit scoring in e-lending platform（2012）中提到，朴素贝叶斯模型可以应用于P2P平台的信用评分系统来预测信贷违约风险。作者提到，朴素贝叶斯模型可以通过借款者的个人信息及人际关系网络信息的输入，对借款者的违约风险进行判定。由于违约人在数据上的相似性，朴素贝叶斯模型对违约风险的判定表现较好。
朴素贝叶斯的优点在于模型本身构造简单，易于解释，且节约运算资源，能够充分考量历史信息。朴素贝叶斯在无冗余特征，预测值与历史分类数据呈现较高相似性，要求较低精度的分类问题中表现较好。
朴素贝叶斯模型的缺点在于，其本身拥有一个较强的理论假设（样本独立同分布），且会以同样的比重考量所有特征，因此在拥有大量相关度较弱的特征，或是数据本身拥有大量异常值时表现不佳。
对于当前问题，不能毕业的学生在特性表现上会体现出相似性，且历史数据对于学生能否毕业的判断占有重要地位，因此可以采用朴素贝叶斯算法。

在Paul Horton的论文Better Prediction of Protein Cellular Localization Sites with the k Nearest Neighbors Classifier（1997）中提到，KNN模型可以用于细胞中蛋白质位置的测定。在预测细胞作用机理时，往往需要对一些特定位置尚未观测到的蛋白质成分进行预测。作者在文中使用了决策树，朴素贝叶斯和KNN模型对蛋白质位置进行预测，结果发现KNN模型表现最好。
KNN模型的优势在于其本身是一个惰性学习器（lazy learner）且模型简单，因此不需要占用资源进行训练，可以直接进行预测。其在低维度数据上表现良好，也可以轻松解决多类分类问题。
KNN模型的缺陷在于，如果数据样本维度太高，预测时容易导致“维度灾难”，使得预测占用大量计算资源，甚至无法预测。
目前的数据量可以尝试使用KNN模型进行预测，且由于KNN惰性学习的特性，校方可以仅在预测时运行干预程序，即插即用，用完即走。

### 准备
运行下面的代码单元以初始化三个帮助函数，这三个函数将能够帮你训练和测试你上面所选择的三个监督学习算法。这些函数是：
- `train_classifier` - 输入一个分类器和训练集，用数据来训练这个分类器。
- `predict_labels` - 输入一个训练好的分类器、特征以及一个目标标签，这个函数将帮你做预测并给出F<sub>1</sub>的值.
- `train_predict` - 输入一个分类器以及训练集和测试集，它可以运行`train_clasifier`和`predict_labels`.
 - 这个函数将分别输出训练集的F<sub>1</sub>值和测试集的F<sub>1</sub>值

In [7]:
def train_classifier(clf, X_train, y_train):
    ''' 用训练集训练分类器 '''
    
    # 开始计时，训练分类器，然后停止计时
    start = time()
    clf.fit(X_train, y_train)
    end = time()
    
    # Print the results
    print "Trained model in {:.4f} seconds".format(end - start)

    
def predict_labels(clf, features, target):
    ''' 用训练好的分类器做预测并输出F1值'''
    
    # 开始计时，作出预测，然后停止计时
    start = time()
    y_pred = clf.predict(features)
    end = time()
    
    # 输出并返回结果
    print "Made predictions in {:.4f} seconds.".format(end - start)
    return f1_score(target.values, y_pred, pos_label='yes')


def train_predict(clf, X_train, y_train, X_test, y_test):
    ''' 用一个分类器训练和预测，并输出F1值 '''
    
    # 输出分类器名称和训练集大小
    print "Training a {} using a training set size of {}. . .".format(clf.__class__.__name__, len(X_train))
    
    # 训练一个分类器
    train_classifier(clf, X_train, y_train)
    
    # 输出训练和测试的预测结果
    print "F1 score for training set: {:.4f}.".format(predict_labels(clf, X_train, y_train))
    print "F1 score for test set: {:.4f}.".format(predict_labels(clf, X_test, y_test))

### 练习: 模型评价指标
借助于上面定义的函数，你现在需要导入三个你选择的监督学习模型，然后为每一个模型运行`train_predict`函数。请记住，对于每一个模型你需要在不同大小的训练集（100，200和300）上进行训练和测试。所以，你在下面应该会有9个不同的输出（每个模型都有训练集大小不同的三个输出）。在接下来的代码单元中，你将需要实现以下功能：
- 引入三个你在上面讨论过的监督式学习算法模型。
- 初始化三个模型并将它们存储在`clf_A`， `clf_B` 和 `clf_C`中。
 - 如果可能对每一个模型都设置一个`random_state`。
 - **注意:** 这里先使用每一个模型的默认参数，在接下来的部分中你将需要对某一个模型的参数进行调整。
- 创建不同大小的训练集用来训练每一个模型。
 - *不要再混洗和再分割数据！新的训练集要取自`X_train`和`y_train`.*
- 对于每一个模型要用不同大小的训练集来训练它，然后在测试集上做测试（总共需要9次训练测试）   
**注意:** 在下面的代码单元后面我们提供了三个表用来存储你的结果。

In [8]:
# TODO：从sklearn中引入三个监督学习模型
# from sklearn import model_A
# from sklearn import model_B
# from skearln import model_C
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier

# TODO：初始化三个模型
clf_A = SVC(kernel='linear')
clf_B = GaussianNB()
clf_C = KNeighborsClassifier()

# TODO：设置训练集大小
# X_train_100 = None
# y_train_100 = None

X_train_100, X_test_100, y_train_100, y_test_100 = train_test_split(X_all, y_all, train_size=100, random_state=1)

# X_train_200 = None
# y_train_200 = None

X_train_200, X_test_200, y_train_200, y_test_200 = train_test_split(X_all, y_all, train_size=200, random_state=2)

# X_train_300 = None
# y_train_300 = None

X_train_300, X_test_300, y_train_300, y_test_300 = train_test_split(X_all, y_all, train_size=300, random_state=3)

# TODO：对每一个分类器和每一个训练集大小运行'train_predict' 
# train_predict(clf, X_train, y_train, X_test, y_test)
train_predict(clf_A, X_train_100, y_train_100, X_test_100, y_test_100)
print '\n'
train_predict(clf_A, X_train_200, y_train_200, X_test_200, y_test_200)
print '\n'
train_predict(clf_A, X_train_300, y_train_300, X_test_300, y_test_300)
print '\n'
train_predict(clf_B, X_train_100, y_train_100, X_test_100, y_test_100)
print '\n'
train_predict(clf_B, X_train_200, y_train_200, X_test_200, y_test_200)
print '\n'
train_predict(clf_B, X_train_300, y_train_300, X_test_300, y_test_300)
print '\n'
train_predict(clf_C, X_train_100, y_train_100, X_test_100, y_test_100)
print '\n'
train_predict(clf_C, X_train_200, y_train_200, X_test_200, y_test_200)
print '\n'
train_predict(clf_C, X_train_300, y_train_300, X_test_300, y_test_300)

Training a SVC using a training set size of 100. . .
Trained model in 0.0150 seconds
Made predictions in 0.0000 seconds.
F1 score for training set: 0.9173.
Made predictions in 0.0000 seconds.
F1 score for test set: 0.7350.


Training a SVC using a training set size of 200. . .
Trained model in 0.0160 seconds
Made predictions in 0.0160 seconds.
F1 score for training set: 0.8780.
Made predictions in 0.0000 seconds.
F1 score for test set: 0.7885.


Training a SVC using a training set size of 300. . .
Trained model in 0.0470 seconds
Made predictions in 0.0000 seconds.
F1 score for training set: 0.8413.
Made predictions in 0.0000 seconds.
F1 score for test set: 0.7794.


Training a GaussianNB using a training set size of 100. . .
Trained model in 0.0000 seconds
Made predictions in 0.0000 seconds.
F1 score for training set: 0.5833.
Made predictions in 0.0000 seconds.
F1 score for test set: 0.4759.


Training a GaussianNB using a training set size of 200. . .
Trained model in 0.0000 seconds
M

### 结果表格
编辑下面的表格看看在[Markdown](https://github.com/adam-p/markdown-here/wiki/Markdown-Cheatsheet#tables)中如何设计一个表格。你需要把上面的结果记录在表格中。

** 分类器 1 - Support Vector Machine（线性内核）**  

| 训练集大小 | 训练时间 | 预测时间 (测试) | F1值 (训练) | F1值 (测试) |
| :---------------: | :---------------------: | :--------------------: | :--------------: | :-------------: |
| 100               |      0.0150            |      0.0000                  |    0.9173              |   0.7350              |
| 200               |      0.0160            |      0.0000                  |    0.8780              |   0.7885              |
| 300               |      0.0470            |      0.0000                  |    0.8413              |   0.7794              |

** 分类器 2 - 朴素贝叶斯**  

| 训练集大小 | 训练时间 | 预测时间 (测试) | F1值 (训练) | F1值 (测试) |
| :---------------: | :---------------------: | :--------------------: | :--------------: | :-------------: |
| 100               |     0.0000                    |     0.0000                   |   0.5833               |   0.4759              |
| 200               |     0.0000                    |     0.0000                   |   0.8129               |   0.7294              |
| 300               |     0.0150                    |     0.0000                   |   0.7912               |   0.7576              |

** 分类器 3 - KNN**  

| 训练集大小 | 训练时间 | 预测时间 (测试) | F1值 (训练) | F1值 (测试) |
| :---------------: | :---------------------: | :--------------------: | :--------------: | :-------------: |
| 100               |     0.0000                    |     0.0000                   |    0.8356              |   0.7534              |
| 200               |     0.0000                    |     0.0160                   |    0.8800              |   0.7746              |
| 300               |     0.0000                    |     0.0000                   |    0.8440              |   0.7943              |

## 选择最佳模型
在最后这一部分中，你将从三个监督学习模型中选择一个用在学生数据上的最佳模型。然后你将在最佳模型上用全部的训练集（`X_train`和`y_train`）运行一个网格搜索算法，在这个过程中，你要至少调整一个参数以提高模型的F<sub>1</sub>值（相比于没有调参的模型的分值有所提高）。 

### 问题 3 - 选择最佳模型
*给予你上面做的实验，用一到两段话，向（学校）监事会解释你将选择哪个模型作为最佳的模型。哪个模型在现有的数据，有限的资源、开支和模型表现综合来看是最好的选择？*

**回答: **本次模型测试使用了SVM，朴素贝叶斯和KNN三个模型。从模型的训练及预测时间来看，朴素贝叶斯最节省资源，KNN其次，SVM资源开销最大。从模型的表现来看，KNN模型表现最佳，SVM模型表现其次，朴素贝叶斯模型表现垫底。由于KNN算法在资源占用上与朴素贝叶斯算法相差不大，且模型表现最好，故应选择KNN模型作为干预系统的模型。

### 问题 4 - 用通俗的语言解释模型
*用一到两段话，向（学校）监事会用外行也听得懂的话来解释最终模型是如何工作的。你需要解释所选模型的主要特点。例如，这个模型是怎样被训练的，它又是如何做出预测的。避免使用高级的数学或技术术语，不要使用公式或特定的算法名词。*

**回答: **干预系统所采用的模型叫K最近邻模型。该模型利用已经被记录为毕业和不能毕业的学生的历史数据进行训练。由于不能毕业的学生在各项属性上具有一定的相似性，比如都拥有相似的学习时间和缺席率，因此这些学生的历史数据可以与需要预测的学生数据进行比较，如果一个学生的数据显示出与历史上不能毕业的学生的数据有较高的相似度，则这个学生不能毕业的概率将会提高，系统更有可能将这个学生标记为不能毕业。

### 练习: 模型调参
细调选择的模型的参数。使用网格搜索（`GridSearchCV`）来至少调整模型的重要参数（至少调整一个），这个参数至少需给出并尝试3个不同的值。你要使用整个训练集来完成这个过程。在接下来的代码单元中，你需要实现以下功能：
- 导入 [`sklearn.model_selection.GridSearchCV`](http://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html) 和 [`sklearn.metrics.make_scorer`](http://scikit-learn.org/stable/modules/generated/sklearn.metrics.make_scorer.html).
- 创建一个对于这个模型你希望调整参数的字典。
 - 例如: `parameters = {'parameter' : [list of values]}`。
- 初始化你选择的分类器，并将其存储在`clf`中。
- 使用`make_scorer` 创建F<sub>1</sub>评分函数并将其存储在`f1_scorer`中。
 - 需正确设定参数`pos_label`的值！
- 在分类器`clf`上用`f1_scorer` 作为评价函数运行网格搜索,并将结果存储在`grid_obj`中。
- 用训练集(`X_train`, `y_train`)训练grid search object,并将结果存储在`grid_obj`中。

In [18]:
# TODO: 导入 'GridSearchCV' 和 'make_scorer'
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import make_scorer
from sklearn.metrics import f1_score
# TODO：创建你希望调整的参数列表
parameters = {'n_neighbors': range(1, 11), 'weights': ['uniform', 'distance']}

# TODO：初始化分类器
clf = KNeighborsClassifier()

# TODO：用'make_scorer'创建一个f1评分函数
f1_scorer = make_scorer(f1_score, pos_label='yes')

# TODO：在分类器上使用f1_scorer作为评分函数运行网格搜索
grid_obj = GridSearchCV(clf, parameters, scoring=f1_scorer)

# TODO: Fit the grid search object to the training data and find the optimal parameters
# TODO：用训练集训练grid search object来寻找最佳参数
grid_obj.fit(X_train, y_train)

# Get the estimator
# 得到预测的结果
clf = grid_obj.best_estimator_

# Report the final F1 score for training and testing after parameter tuning
# 输出经过调参之后的训练集和测试集的F1值
print "Tuned model has a training F1 score of {:.4f}.".format(predict_labels(clf, X_train, y_train))
print "Tuned model has a testing F1 score of {:.4f}.".format(predict_labels(clf, X_test, y_test))

Made predictions in 0.0060 seconds.
Tuned model has a training F1 score of 0.8349.
Made predictions in 0.0030 seconds.
Tuned model has a testing F1 score of 0.8108.


### 问题 5 - 最终的 F<sub>1</sub> 值
*最终模型的训练和测试的F<sub>1</sub>值是多少？这个值相比于没有调整过参数的模型怎么样？*

**回答: **最终模型训练F1值为0.8349，测试F1值为0.8108，相比于默认模型有所提高。

> **注意**: 当你写完了所有的代码，并且回答了所有的问题。你就可以把你的 iPython Notebook 导出成 HTML 文件。你可以在菜单栏，这样导出**File -> Download as -> HTML (.html)**把这个 HTML 和这个 iPython notebook 一起做为你的作业提交。  